<a href="https://colab.research.google.com/github/ijnuegnaj/RAG_PRACTICE/blob/practice8/BM25_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain openai tiktoken langchain-community rank_bm25 sentence-transformers chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 11.9 MB/s eta 0:0

In [ ]:
import openai, tiktoken
#재귀적 텍스트 분할
from langchain.text_splitter import RecursiveCharacterTextSplitter
#웹주소로 데이터를 load
from langchain.document_loaders import WebBaseLoader
#허깅페이스임베딩, BM25, 앙상블 리트리버
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain.vectorstores import Chroma

In [ ]:
#토크나이저 만들기
#cl100k_base : GPT와 유사한 형식의 토크나이저를 활용
tokenizer = tiktoken.get_encoding('cl100k_base')
def tiktoken_len(text):
  tokens = tokenizer.encode(text)
  return len(tokens)

In [ ]:
model_hug = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')

/tmp/ipython-input-2552533613.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model_hug = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
loaders=[
    WebBaseLoader('https://en.wikipedia.org/wiki/Retrieval-augmented_generation'),
    WebBaseLoader('https://en.wikipedia.org/wiki/Fine-tuning_(deep_learning)')
]

In [ ]:
docs = []
for load in loaders:
  docs.extend(load.load_and_split())

In [ ]:
#청크화
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    length_function=tiktoken_len
)
texts = text_splitter.split_documents(docs)

In [ ]:
#1.bm25_retriever, 유사도가 가장 높은 3개의 답변을 return함
bm25_r = BM25Retriever.from_documents(texts)
bm25_r.k = 3

In [ ]:
#2.chorma vector 리트리버, 유사도가 가장 높은 3개의 답변을 return
chroma_r = Chroma.from_documents(texts, model_hug)
chroma_r = chroma_r.as_retriever(search_kwargs={'k':3})

In [ ]:
#앙상블
ensemble = EnsembleRetriever(retrievers=[bm25_r, chroma_r])

docs = ensemble.invoke('difference between fine-tuning and RAG')

In [ ]:
docs[0]

Document(metadata={'source': 'https://en.wikipedia.org/wiki/Fine-tuning_(deep_learning)', 'title': 'Fine-tuning (deep learning) - Wikipedia', 'language': 'en'}, page_content='Representation fine-tuning[edit]\nThis section relies largely or entirely upon a single source. Relevant discussion may be found on the talk page. Please help improve this article by introducing citations to additional sources at this section.Find sources:\xa0"Representation fine-tuning (ReFT)"\xa0‚Äì\xa0news\xa0¬∑ newspapers\xa0¬∑ books\xa0¬∑ scholar\xa0¬∑ JSTOR (May 2024) (Learn how and when to remove this message)\nRepresentation fine-tuning (ReFT) is a technique developed by researchers at Stanford University aimed at fine-tuning large language models (LLMs) by modifying less than 1% of their representations. Unlike parameter-efficient fine-tuning (PEFT) methods, which mainly focus on updating weights, ReFT targets representations, suggesting that modifying representations might be a more effective strategy th

In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] =  userdata.get('ssu')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI()
question = 'What is the top 3 difference between fine-tuning and RAG?'
qa_chain = RetrievalQA.from_chain_type(llm,
                                       chain_type='stuff',
                                       retriever=ensemble)
result = qa_chain.run(question)
print(result)

NameError: name 'ensemble' is not defined

In [ ]:
!pip install tavily-python

In [ ]:
!pip install langchain-openai

In [ ]:
#정해진 양식으로 input을 넣을 수 있도록 조절
from langchain_core.prompts import ChatPromptTemplate
#중요!!! pydantic, typing => 데이터 타입을 정의하는것을 도와주는 라이브러리..
#LLM, 딥러닝 '서빙' (서비스화)하는 과정에서 인터넷을 통해 데이터를 주고받음
#데이터의 '데이터 타입'이 원했던 타입이 아니라면 -> 오류 발생 가능성이 높다
#pydantic, typing을 통해서 내가 받기를 원하는 데이터의 데이터 타입을 미리 지정
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.chat_models import ChatOpenAI

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#pydantic의 BaseModel을 상속받아서 포맷팅
class GradeDocuments(BaseModel):
  binary_score : str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
  )

In [ ]:
#실제 대화를 구성
llm = ChatOpenAI(model_name = 'gpt-3.5-turbo')
#구조화된 llm > LLM 응답을 지정된 구조로 파싱/검증하는 기능
structured_llm_grader = llm.with_structured_output(GradeDocuments)
system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', system),
        ('human', 'Retrieved Documents : {document}, User Question : {question}')
    ]
)

/tmp/ipython-input-3978341730.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name = 'gpt-3.5-turbo')


ValidationError: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_name': 'gpt-3.5-t...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error